In [62]:
import requests
from bs4 import BeautifulSoup

In [63]:
url = 'https://auth.realmassive.com/login'

payload = { 
         
    "email": "samuel.wondim@gmail.com", 
    "password": "Testing125!"
          }

In [64]:
s = requests.Session()
p = s.post(url, data=payload)

In [65]:
p.status_code

400

In [66]:
html = p.content
soup = BeautifulSoup(html, 'html.parser')
print(soup.prettify())
s.close()

{"status": "error", "message": "Missing required param(s): {'password', 'email'}"}



In [67]:
with open('rmdata.html', 'wb') as file:
    file.write(soup.prettify('utf-8'))